In [ ]:
import os
from pathlib import Path
import datetime 

import numpy as np
import pandas as pd
from astropy.table import Table, join, hstack, vstack
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic
import matplotlib.dates as md
from scipy.integrate import trapezoid

from utils import better_step

In [ ]:
params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
    "figure.facecolor": "w",
    "xtick.top": True,
    "ytick.right": True,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "font.family": "serif",
    "mathtext.fontset": "dejavuserif",
}
plt.rcParams.update(params)

In [ ]:
base_path = Path(os.environ["CFS"]) / "desi" / "users" / "bid13" / "DESI_II"
deep_path = base_path / "other_surveys" / "DEEP23"

In [ ]:
hsc_groth_cat = Table.read(deep_path / "HSC_GROTH.fits").to_pandas()
deep_spec_cat = Table.read(deep_path / "alldeep.egs.2012jun13.fits.gz").to_pandas()
deep_spec_cat = deep_spec_cat[deep_spec_cat["ZQUALITY"]>0]
deep_spec_cat = deep_spec_cat[deep_spec_cat["Z"]>0.01]

hsc_cosmos_cat = Table.read(base_path / "target_data" / "HSC_COSMOS_I_mag_lim_24.8.fits").to_pandas()
zcosmos_spec_cat = Table.read(base_path / "other_surveys" / "ZCOSMOS" / "zcosmos-bright.fits").to_pandas()
zcosmos_spec_cat = zcosmos_spec_cat[zcosmos_spec_cat["REDSHIFT"]>0.01]


vi_results = pd.read_csv(Path(os.environ["SCRATCH"]) / "DESI-II-VI-results" / "merged_vi_resolved.csv" )
desi_spec_cat = Table.read(Path(os.environ["CFS"]) / "desi/users/bid13/DESI_II/pilot_obs/merged_cat.fits")
desi_spec_cat = desi_spec_cat[desi_spec_cat["Z"]>0.01]

cat_desi = join(Table.from_pandas(vi_results),desi_spec_cat,keys="TARGETID",join_type="inner")
cat_desi["success"] = (cat_desi["VI_quality"]>2)
cat_desi = vstack([cat_desi[~cat_desi["success"]],
                      cat_desi[cat_desi["success"] & (cat_desi["Z"]>0.01)]])


In [ ]:
def flux_to_mag(flux):
    return -2.5*np.log10(flux*1e-9) + 8.90

In [ ]:
def process_hsc_cat(hsc_cat):
    hsc_cat["mag_i"] = flux_to_mag(hsc_cat["i_cmodel_flux"])-hsc_cat["a_i"]

    hsc_cat["mag_i_fiber"] = flux_to_mag(hsc_cat["i_fiber_flux"])-hsc_cat["a_i"]
    
    ## Quality cuts
    # valid I-band flux
    qmask = np.isfinite(hsc_cat["i_cmodel_flux"]) & (hsc_cat["i_cmodel_flux"]>0)
    #cmodel fit not failed
    qmask &= (~hsc_cat["i_cmodel_flag"].values)
    #General Failure Flag
    qmask &= (~hsc_cat["i_sdsscentroid_flag"].values)

    # Possible cuts: Bright objects nearby, bad pixels

    #star-galaxy separation (is point source in I band)
    # mask &= (hsc_cat["i_extendedness_value"]>0)


    i_min = 22
    i_max = 24.5
    mask = (hsc_cat["mag_i"] <i_max) & (hsc_cat["mag_i"] >i_min)
    i_fiber_min = 22
    i_fiber_max = 24.75
    mask &= (hsc_cat["mag_i_fiber"] <i_fiber_max) & (hsc_cat["mag_i_fiber"] >i_fiber_min)
    
    return hsc_cat[qmask & mask]

In [ ]:
hsc_groth_cat = process_hsc_cat(hsc_groth_cat)
hsc_coord = SkyCoord(ra=hsc_groth_cat["ra"].values*u.degree, dec=hsc_groth_cat["dec"].values*u.degree)
deep_coord = SkyCoord(ra=deep_spec_cat["RA"].values*u.degree, dec=deep_spec_cat["DEC"].values*u.degree)

idx, d2d, d3d = deep_coord.match_to_catalog_sky(hsc_coord)
phot_cat = hsc_groth_cat.iloc[idx]

idx_mask = d2d.arcsec<1


cat_deep = pd.concat([deep_spec_cat.reset_index(),phot_cat.reset_index()], axis=1)

cat_deep = cat_deep[idx_mask]
print(len(cat_deep)/len(deep_coord))

In [ ]:
hsc_cosmos_cat = process_hsc_cat(hsc_cosmos_cat)
hsc_coord = SkyCoord(ra=hsc_cosmos_cat["ra"].values*u.degree, dec=hsc_cosmos_cat["dec"].values*u.degree)
zcosmos_coord = SkyCoord(ra=zcosmos_spec_cat["RAJ2000"].values*u.degree, dec=zcosmos_spec_cat["DEJ2000"].values*u.degree)

idx, d2d, d3d = zcosmos_coord.match_to_catalog_sky(hsc_coord)
phot_cat = hsc_cosmos_cat.iloc[idx]

idx_mask = d2d.arcsec<1


cat_zcosmos = pd.concat([zcosmos_spec_cat.reset_index(),phot_cat.reset_index()], axis=1)

cat_zcosmos = cat_zcosmos[idx_mask]
print(len(cat_zcosmos)/len(zcosmos_coord))

In [ ]:
cat_deep["success"] = cat_deep["ZQUALITY"]>2
###Figure out how to merge VI spectypes
cat_deep = cat_deep[cat_deep["Z"]>0.01]
cat_zcosmos["success"] = np.isin(cat_zcosmos["CC"].astype(int), [3,4,13,14,23,24,213,214])
cat_zcosmos = cat_zcosmos[cat_zcosmos["REDSHIFT"]>0.01]

In [ ]:
def make_success_plot(cat,colname="mag_i", nbins=10, ref_sample=None, ax =None, **kwargs):
    success, bin_edges , _ = binned_statistic(cat[colname], cat["success"],bins=nbins,statistic="mean")
    count, bin_edges, _ = binned_statistic(cat[colname], cat["success"],bins=nbins,statistic="count")
    adjustment =1.0
    if ref_sample is not None:
        for i in range(len(bin_edges)-1):
            sel_cat = ref_sample[(ref_sample["HSC_i_MAG"]>=bin_edges[i]) & (ref_sample["HSC_i_MAG"]<bin_edges[i])]
            frac_greater = np.sum(sel_cat["Z"]>1.6)/len(sel_cat)
            print(frac_greater)
    e_success = np.sqrt(success*(1-success)/(count*adjustment))
    ax = better_step(bin_edges, success, yerr=e_success, ax = ax,**kwargs)
    # ax.set_ylim(0.3,1)
    # ax.grid(linestyle="--")
    return ax

### Success of all objects

In [ ]:

ax = make_success_plot(cat_desi)
ax = make_success_plot(cat_deep)
ax = make_success_plot(cat_zcosmos, ax=ax)
ax.set_title("All Objects")
ax.set_xlabel("$i$-mag")

In [ ]:
ax = make_success_plot(cat_desi, colname="mag_i_fiber")
ax = make_success_plot(cat_deep, colname="mag_i_fiber")
ax = make_success_plot(cat_zcosmos, ax=ax, colname="mag_i_fiber")
ax.set_title("All Objects")
ax.set_xlabel("$i$-fiber-mag")

### Success in bins of magnitude

In [ ]:
mag_bin_edges = [22,22.5,23,23.75, 24.5]
fig, axs = plt.subplots(2,2, figsize=(14,12))
axs = np.ravel(axs)

for i in range(4):
    cat_sel = cat_desi[(cat_desi["mag_i"]>=mag_bin_edges[i]) & (cat_desi["mag_i"]<mag_bin_edges[i+1])]
    make_success_plot(cat_sel, ax= axs[i])
    
    
    cat_sel = cat_deep[(cat_deep["mag_i"]>=mag_bin_edges[i]) & (cat_deep["mag_i"]<mag_bin_edges[i+1])]
    make_success_plot(cat_sel, ax= axs[i])
    
    cat_sel = cat_zcosmos[(cat_zcosmos["mag_i"]>=mag_bin_edges[i]) & (cat_zcosmos["mag_i"]<mag_bin_edges[i+1])]
    make_success_plot(cat_sel, ax= axs[i])
    
    axs[i].set_title(f"{mag_bin_edges[i]}$\leq i <${mag_bin_edges[i+1]}")
    # axs[i].axhline(np.mean(cat_sel["success"]), ls="--", c="k", alpha=0.5)
    axs[i].set_xlabel("$i$-mag")

In [ ]:
# mag_bin_edges = [22,22.75,23.25,24, 24.75]
mag_bin_edges = [22,22.5,23,23.75, 24.5]
fig, axs = plt.subplots(2,2, figsize=(14,12))
axs = np.ravel(axs)

for i in range(4):
    cat_sel = cat_desi[(cat_desi["mag_i_fiber"]>=mag_bin_edges[i]) & (cat_desi["mag_i_fiber"]<mag_bin_edges[i+1])]
    make_success_plot(cat_sel, ax= axs[i],colname="mag_i_fiber")
    
    
    cat_sel = cat_deep[(cat_deep["mag_i_fiber"]>=mag_bin_edges[i]) & (cat_deep["mag_i_fiber"]<mag_bin_edges[i+1])]
    make_success_plot(cat_sel, ax= axs[i],colname="mag_i_fiber")
    
    cat_sel = cat_zcosmos[(cat_zcosmos["mag_i_fiber"]>=mag_bin_edges[i]) & (cat_zcosmos["mag_i_fiber"]<mag_bin_edges[i+1])]
    make_success_plot(cat_sel, ax= axs[i],colname="mag_i_fiber")
    # axs[i].axhline(np.mean(cat_sel["success"]), ls="--", c="k", alpha=0.5)
    axs[i].set_xlabel("$i$-fiber-mag")

### Success in bins of time as a function of magnitude

In [ ]:
mag_bin_edges = [22,22.5,23,23.75, 24.5]
fig, axs = plt.subplots(1,3, figsize=(17,4.5))
axs = np.ravel(axs)

# cat_list = [cat_desi, cat_deep, cat_zcosmos]
# for cat in cat_list:
cat_sel = cat_desi[(cat_desi["EXPTIME"]<=1.25*3600)]
make_success_plot(cat_sel, ax= axs[0],label="DESI")
make_success_plot(cat_deep, ax= axs[0], label="DEEP-2/3", ls="--")
make_success_plot(cat_zcosmos, ax= axs[0],label="ZCOSMOS-Bright", ls="--")
axs[0].set_title("25m to 1h15m", fontsize=20)
axs[0].legend(frameon=False)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())
print(np.median(cat_sel["EXPTIME"]))

cat_sel = cat_desi[(cat_desi["EXPTIME"]>1.25*60) & (cat_desi["EXPTIME"]<=3.75*3600)]
make_success_plot(cat_sel, ax= axs[1])
make_success_plot(cat_deep, ax= axs[1], ls="--")
make_success_plot(cat_zcosmos, ax= axs[1], ls="--")
axs[1].set_title("1h15m to 3h45m", fontsize=20)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())
print(np.median(cat_sel["EXPTIME"]))

cat_sel = cat_desi[(cat_desi["EXPTIME"]>3.75*3600)]
make_success_plot(cat_sel, ax= axs[2])
make_success_plot(cat_deep, ax= axs[2], ls="--")
make_success_plot(cat_zcosmos, ax= axs[2], ls="--")
axs[2].set_title("3h45m to 8h", fontsize=20)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())
print(np.median(cat_sel["EXPTIME"]))

axs[0].set_ylabel("Success Rate", fontsize=25)
axs[1].set_xlabel("$i$-mag", fontsize=25)
plt.savefig("i_mag_success.pdf", bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(6,4.5))
cat_sel = cat_desi[(cat_desi["EXPTIME"]>1.25*60) & (cat_desi["EXPTIME"]<=3.75*3600)]
make_success_plot(cat_sel, ax= axs,label="DESI")
make_success_plot(cat_deep, ax= axs, ls="--", label="Keck (10m)-based survey")
axs.set_ylabel("Success Rate", fontsize=25)
axs.set_xlabel("$i$-mag", fontsize=25)
axs.set_ylim(0.1,1)
axs.legend(loc="lower left",frameon=False, fontsize=18)
plt.savefig("p5-desi-faint.png",dpi=300, bbox_inches='tight')

In [ ]:
mag_bin_edges = [22,22.5,23,23.75, 24.5]
fig, axs = plt.subplots(1,3, figsize=(17,4.5))
axs = np.ravel(axs)

# cat_list = [cat_desi, cat_deep, cat_zcosmos]
# for cat in cat_list:
cat_sel = cat_desi[(cat_desi["EXPTIME"]<=1.25*3600)]
make_success_plot(cat_sel, ax= axs[0],label="DESI", colname="mag_i_fiber")
make_success_plot(cat_deep, ax= axs[0], label="DEEP-2/3", colname="mag_i_fiber", ls="--")
make_success_plot(cat_zcosmos, ax= axs[0],label="ZCOSMOS-Bright", colname="mag_i_fiber", ls="--")
axs[0].set_title("25m to 1h15m", fontsize=20)
axs[0].legend(frameon=False)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())

cat_sel = cat_desi[(cat_desi["EXPTIME"]>1.25*60) & (cat_desi["EXPTIME"]<=3.75*3600)]
make_success_plot(cat_sel, ax= axs[1], colname="mag_i_fiber")
make_success_plot(cat_deep, ax= axs[1], colname="mag_i_fiber", ls="--")
make_success_plot(cat_zcosmos, ax= axs[1], colname="mag_i_fiber", ls="--")
axs[1].set_title("1h15m to 3h45m", fontsize=20)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())

cat_sel = cat_desi[(cat_desi["EXPTIME"]>3.75*3600)]
make_success_plot(cat_sel, ax= axs[2], colname="mag_i_fiber")
make_success_plot(cat_deep, ax= axs[2], colname="mag_i_fiber", ls="--")
make_success_plot(cat_zcosmos, ax= axs[2], colname="mag_i_fiber", ls="--")
axs[2].set_title("3h45m to 8h", fontsize=20)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())

axs[0].set_ylabel("Success Rate", fontsize=25)
axs[1].set_xlabel("$i$-fiber-mag", fontsize=25)
plt.savefig("i_fiber_mag_success.pdf", bbox_inches='tight')

### Treat z>1.45 as incorrect

In [ ]:
names = [name for name in cat_desi.colnames if len(cat_desi[name].shape) <= 1]
cat_desi_short = cat_desi[names].to_pandas()
# cat_desi_short = cat_desi.copy()
cat_desi_short.loc[cat_desi_short["Z"]>1.45,"success"] = False
mag_bin_edges = [22,22.5,23,23.75, 24.5]
fig, axs = plt.subplots(1,3, figsize=(17,4.5))
axs = np.ravel(axs)

# cat_list = [cat_desi, cat_deep, cat_zcosmos]
# for cat in cat_list:
cat_sel = cat_desi_short[(cat_desi_short["EXPTIME"]<=1.25*3600)]
make_success_plot(cat_sel, ax= axs[0],label="DESI")
make_success_plot(cat_deep, ax= axs[0], label="DEEP-2/3", ls="--")
make_success_plot(cat_zcosmos, ax= axs[0],label="ZCOSMOS-Bright", ls="--")
axs[0].set_title("25m to 1h15m", fontsize=20)
axs[0].legend(frameon=False)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())

cat_sel = cat_desi_short[(cat_desi_short["EXPTIME"]>1.25*60) & (cat_desi_short["EXPTIME"]<=3.75*3600)]
make_success_plot(cat_sel, ax= axs[1])
make_success_plot(cat_deep, ax= axs[1], ls="--")
make_success_plot(cat_zcosmos, ax= axs[1], ls="--")
axs[1].set_title("1h15m to 3h45m", fontsize=20)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())

cat_sel = cat_desi_short[(cat_desi_short["EXPTIME"]>3.75*3600)]
make_success_plot(cat_sel, ax= axs[2])
make_success_plot(cat_deep, ax= axs[2], ls="--")
make_success_plot(cat_zcosmos, ax= axs[2], ls="--")
axs[2].set_title("3h45m to 8h", fontsize=20)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())

axs[0].set_ylabel("Success Rate", fontsize=25)
axs[1].set_xlabel("$i$-mag", fontsize=25)
# fig.suptitle("*considering $z>1.45$ as a failure", y=1.05, fontsize=15)
plt.savefig("i_mag_success_DEEP_criteria.pdf", bbox_inches='tight')

In [ ]:
mag_bin_edges = [22,22.5,23,23.75, 24.5]
fig, axs = plt.subplots(1,3, figsize=(17,4.5))
axs = np.ravel(axs)

# cat_list = [cat_desi, cat_deep, cat_zcosmos]
# for cat in cat_list:
cat_sel = cat_desi_short[(cat_desi_short["EXPTIME"]<=1.25*3600)]
make_success_plot(cat_sel, ax= axs[0],label="DESI", colname="mag_i_fiber")
make_success_plot(cat_deep, ax= axs[0], label="DEEP-2/3", colname="mag_i_fiber", ls="--")
make_success_plot(cat_zcosmos, ax= axs[0],label="ZCOSMOS-Bright", colname="mag_i_fiber", ls="--")
axs[0].set_title("25m to 1h15m", fontsize=20)
axs[0].legend(frameon=False)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())

cat_sel = cat_desi_short[(cat_desi_short["EXPTIME"]>1.25*60) & (cat_desi_short["EXPTIME"]<=3.75*3600)]
make_success_plot(cat_sel, ax= axs[1], colname="mag_i_fiber")
make_success_plot(cat_deep, ax= axs[1], colname="mag_i_fiber", ls="--")
make_success_plot(cat_zcosmos, ax= axs[1], colname="mag_i_fiber", ls="--")
axs[1].set_title("1h15m to 3h45m", fontsize=20)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())

cat_sel = cat_desi_short[(cat_desi_short["EXPTIME"]>3.75*3600)]
make_success_plot(cat_sel, ax= axs[2], colname="mag_i_fiber")
make_success_plot(cat_deep, ax= axs[2], colname="mag_i_fiber", ls="--")
make_success_plot(cat_zcosmos, ax= axs[2], colname="mag_i_fiber", ls="--")
axs[2].set_title("3h45m to 8h", fontsize=20)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())

axs[0].set_ylabel("Success Rate", fontsize=25)
axs[1].set_xlabel("$i$-fiber-mag", fontsize=25)
# fig.suptitle("*considering $z>1.45$ as a failure", y=1.05, fontsize=15)
plt.savefig("i_fiber_mag_success_DEEP_criteria.pdf", bbox_inches='tight')

### Success as a function of exposure time in bins of magnitude

In [ ]:
# import matplotlib

# class myFormatter(matplotlib.ticker.Formatter):
#     def __call__(self,in_value_in_seconds,pos=None):
#         hours = int(in_value_in_seconds//(3600))
#         in_value_in_seconds -= hours * 3600
#         mins = int(in_value_in_seconds//(60))
#         in_value_in_seconds -= mins * 60
#         secs = int(in_value_in_seconds)

#         return str(hours) + ":" + str(mins) + ":" + str(secs)

In [ ]:
# mag_bin_edges = [22,22.5,23,23.75, 24.5]
fig, axs = plt.subplots(1,2, figsize=(13,4.5))
axs = np.ravel(axs)

# cat_list = [cat_desi, cat_deep, cat_zcosmos]
# for cat in cat_list:
cat_sel = cat_desi[(cat_desi["mag_i"]<=23)]
cat_sel = cat_desi[(cat_desi["EXPTIME"]<=20000)]
# cat_sel["EXPTIME"] = pd.to_datetime(cat_sel["EXPTIME"],unit="s")
make_success_plot(cat_sel, ax= axs[0],colname="EXPTIME")
# make_success_plot(cat_deep, ax= axs[0], label="DEEP-2/3", ls="--")
# make_success_plot(cat_zcosmos, ax= axs[0],label="ZCOSMOS-Bright", ls="--")
axs[0].set_title("22<$i$-mag$\leq$23", fontsize=20)
# axs[0].legend(frameon=False)
axs[0].set_ylim(0.38,1)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())
axs[0].set_xticks([3600*i for i in range(1,6)], labels=[f"{i}h" for i in range(1,6)])
           
cat_sel = cat_desi[cat_desi["mag_i"]>23]
make_success_plot(cat_sel, ax= axs[1],colname="EXPTIME")
# make_success_plot(cat_deep, ax= axs[1], ls="--")
# make_success_plot(cat_zcosmos, ax= axs[1], ls="--")
axs[1].set_title("23<$i$-mag$\leq$24.5", fontsize=20)
axs[1].set_ylim(0.38,1)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())
axs[1].set_xticks([3600*i for i in range(1,9)], labels=[f"{i}h" for i in range(1,9)])

# for i in range(2):
#     axs[i].xaxis.set_major_formatter(myFormatter)
#     axs[i].xaxis.set_minor_formatter(myFormatter)
#     axs[i].yaxis.set_major_formatter(myFormatter)
#     axs[i].yaxis.set_minor_formatter(myFormatter)

axs[0].set_ylabel("Success Rate", fontsize=25)
fig.text(0.35,0.0,"Exposure Time", fontsize=25)
plt.savefig("exptime_success_imag.pdf", bbox_inches='tight')

In [ ]:
# mag_bin_edges = [22,22.5,23,23.75, 24.5]
fig, axs = plt.subplots(1,2, figsize=(13,4.5))
axs = np.ravel(axs)

# cat_list = [cat_desi, cat_deep, cat_zcosmos]
# for cat in cat_list:
cat_sel = cat_desi[(cat_desi["mag_i_fiber"]<=23)]
cat_sel = cat_desi[(cat_desi["EXPTIME"]<=20000)]
# cat_sel["EXPTIME"] = pd.to_datetime(cat_sel["EXPTIME"],unit="s")
make_success_plot(cat_sel, ax= axs[0],colname="EXPTIME")
# make_success_plot(cat_deep, ax= axs[0], label="DEEP-2/3", ls="--")
# make_success_plot(cat_zcosmos, ax= axs[0],label="ZCOSMOS-Bright", ls="--")
axs[0].set_title("22<$i$-mag-fiber$\leq$23", fontsize=20)
axs[0].legend(frameon=False)
axs[0].set_ylim(0.38,1)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())
axs[0].set_xticks([3600*i for i in range(1,6)], labels=[f"{i}h" for i in range(1,6)])

cat_sel = cat_desi[cat_desi["mag_i_fiber"]>23]
make_success_plot(cat_sel, ax= axs[1],colname="EXPTIME")
# make_success_plot(cat_deep, ax= axs[1], ls="--")
# make_success_plot(cat_zcosmos, ax= axs[1], ls="--")
axs[1].set_title("23<$i$-mag-fiber$\leq$24.5", fontsize=20)
axs[1].set_ylim(0.38,1)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())
axs[1].set_xticks([3600*i for i in range(1,9)], labels=[f"{i}h" for i in range(1,9)])


axs[0].set_ylabel("Success Rate", fontsize=25)
fig.text(0.35,0.0,"Exposure Time", fontsize=25)
plt.savefig("exptime_success_i_fiber_mag.pdf", bbox_inches='tight')

### Other plots

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,4))
ax[0].hist(desi_spec_cat['mag_i'], bins=30, histtype="step", lw=2)
ax[0].set_xlabel("$i$-mag", fontsize=25)

ax[1].hist(desi_spec_cat['mag_i_fiber'], bins=30, histtype="step", lw=2)
ax[1].set_xlabel("$i$-fiber-mag", fontsize=25)
ax[0].set_ylabel("Count", fontsize=25)
plt.savefig("observed_mag_distribution.pdf", bbox_inches='tight')

In [ ]:
plt.hist(cat_desi[cat_desi["success"]]["Z"], bins=30, histtype="step", lw=2)
plt.yscale("log")
plt.xlabel("Redshift", fontsize=25)
plt.ylabel("Count", fontsize=25)
plt.savefig("observed_redshift_distribution.pdf", bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,4))
times = pd.to_datetime(cat_desi["EXPTIME"],unit="s")
ax.hist(times, bins=40, histtype="step", lw=2)
# ax.set_yscale("log")
xfmt = md.DateFormatter('%-Hh')
ax.text(0.021,0.75, "~45m", fontsize=20, transform=ax.transAxes)
ax.text(0.23,0.75, "~1h21m", fontsize=20, transform=ax.transAxes)
ax.text(0.5,0.75, "~4h53m", fontsize=20, transform=ax.transAxes)
ax.xaxis.set_major_formatter(xfmt)
ax.axvline(datetime.datetime.strptime("1970-01-01 01:15:00","%Y-%m-%d %H:%M:%S"), ls = "--", c="grey")
ax.axvline(datetime.datetime.strptime("1970-01-01 03:45:00","%Y-%m-%d %H:%M:%S"), ls = "--", c="grey")
ax.set_xlabel("Exposure Time", fontsize=25)
ax.set_ylabel("Count", fontsize=25)
plt.savefig("observed_exposure_time_distribution.pdf", bbox_inches='tight')

In [ ]:
plt.hist(cat_desi["VI_quality"])
plt.yscale("log")

### Quality Indicator

In [ ]:
rng = np.random.default_rng(42)
noise = rng.uniform(-0.01,0.01, len(cat_desi))
plt.scatter(cat_desi["VI_quality"]+noise,cat_desi["DELTACHI2"], alpha=0.2)
plt.yscale("log")
plt.xlim(-0.3,4.3)
plt.ylim(ymax=1e6)
plt.fill_between(np.linspace(2.8,4.3,10),0,1e6, alpha=0.03,color="k")
plt.xlabel("VI Quality Flag", fontsize=25)
plt.ylabel(r"$\Delta \chi^{2}$", fontsize=30)

In [ ]:
cat_desi = cat_desi[cat_desi["VI_quality"]>-1]
cat_desi.write(base_path / "LSSTY1_VI_cat.fits", overwrite=True)

# z>1.65 pop

In [ ]:
def z0_for_limit(maglimit, band='i'):
    if band=='i':
        z0 = -0.724785  +  0.0408182 * maglimit #21.5 < i < 23 
    elif band=='r':
        z0 = -1.10356 +    0.0567708 * maglimit #for 22 < r < 24
    return z0

In [ ]:
def dNdz(z, z0, alpha ):
    return z**2 * np.exp((-z/z0)**alpha)/(2*z0**3)

In [ ]:
def dNdz_cumulative_mag(z_grid,maglimit,alpha=1,band='i'):
    z0 = z0_for_limit(maglimit, band=band)
    return dNdz(z_grid, z0, alpha=alpha)

In [ ]:
z_grid = np.linspace(0,3,100)

for z in [22,23,24.1,25.3]:
    number_den = dNdz_cumulative_mag(z_grid,z,alpha=1,band='i')
    
    plt.plot(z_grid, number_den, label=f"{z}")
plt.axvline(1.6, color="k", ls ="--")
plt.legend(title=r"$i$-mag",title_fontsize=20, frameon=False)
plt.xlabel(r"$z$", fontsize=20)
plt.ylabel(r"$\frac{1}{N_{total}}\dfrac{dn}{dz}$", fontsize=20)

In [ ]:
base_path = Path("/pscratch/sd/b/bid13/photo-z-desi/redshifts")

spec_cat = Table.read( base_path / "COSMOS2020_FARMER_R1_v2.1_p3.fits").to_pandas()
mask = (spec_cat['FLAG_COMBINED']==0) & ((spec_cat['lp_type']==0) | (spec_cat['lp_type']==2))
spec_cat = spec_cat[mask]
z_list = np.zeros(len(spec_cat))
z_list[(spec_cat['lp_type']==0).to_numpy(dtype=bool)] = spec_cat["lp_zBEST"][(spec_cat['lp_type']==0)]
z_list[(spec_cat['lp_type']==2).to_numpy(dtype=bool)] = spec_cat["lp_zq"][(spec_cat['lp_type']==2)]
spec_cat["Z"] = z_list



In [ ]:
i_grid = [22,23,24.1,25.3]
for idx, i_mag in enumerate(i_grid):
    mask = spec_cat["HSC_i_MAG"]<=i_mag
    sel_cat = spec_cat[mask]
    
    z0 = z0_for_limit(i_mag)
    number_den = dNdz(z_grid, z0, 1)
    number_den /=trapezoid(number_den, z_grid)
    
    plt.plot(z_grid, number_den, color=f"C{idx+1}",ls="--")
    
    plt.hist(sel_cat["Z"], bins=50, histtype="step",lw=2,label=f"{i_mag}", density=True, color=f"C{idx+1}")
plt.legend()
plt.xlim(0,3)

In [ ]:
def make_success_plot(cat,colname="mag_i", nbins=10,analytic_correction=False, ref_sample=None, threshold=1.6, ax =None, **kwargs):
    
    count, bin_edges, _ = binned_statistic(cat[colname], cat["success"],bins=nbins,statistic="count")
    frac_greater_list = []
    if analytic_correction:
        for i in range(len(bin_edges)-1):
            z_grid = np.linspace(threshold,100,5000)
            frac_greater_list.append(trapezoid(dNdz_cumulative_mag(z_grid,bin_edges[i+1])) - trapezoid(dNdz_cumulative_mag(z_grid,bin_edges[i])) )
        # print(frac_greater_list)
            
        success, bin_edges , _ = binned_statistic(cat[colname], cat["success"],bins=nbins,statistic="mean")
        failure = 1-success
        failure = failure - np.array(frac_greater_list)
        success = 1-failure
        e_success = np.sqrt(success*(1-success)/(count))

    elif ref_sample is not None:
        for i in range(len(bin_edges)-1):
            # print(bin_edges[i])
            sel_cat = ref_sample[(ref_sample["HSC_i_MAG"]>=bin_edges[i]) & (ref_sample["HSC_i_MAG"]<bin_edges[i+1])]
            frac_greater_list.append( np.sum(sel_cat["Z"]>threshold)/len(sel_cat) )
        # print(frac_greater_list)
        success, bin_edges , _ = binned_statistic(cat[colname], cat["success"],bins=nbins,statistic="mean")
        failure = 1-success
        failure = failure - np.array(frac_greater_list)
        success = 1-failure
        e_success = np.sqrt(success*(1-success)/(count))
    else:
        success, bin_edges , _ = binned_statistic(cat[colname], cat["success"],bins=nbins,statistic="mean")
        e_success = np.sqrt(success*(1-success)/(count))
    ax = better_step(bin_edges, success, yerr=e_success, ax = ax,**kwargs)
    # ax.set_ylim(0.3,1)
    # ax.grid(linestyle="--")
    return ax

In [ ]:
mag_bin_edges = [22,22.5,23,23.75, 24.5]
fig, axs = plt.subplots(1,3, figsize=(17,4.5))
z_grid = np.linspace(1.6,100,5000)

axs = np.ravel(axs)

# cat_list = [cat_desi, cat_deep, cat_zcosmos]
# for cat in cat_list:
cat_sel = cat_desi[(cat_desi["EXPTIME"]<=1.25*3600)]
make_success_plot(cat_sel, ax= axs[0],label="DESI-Adjusted by data", ref_sample=spec_cat, threshold=1.6)
make_success_plot(cat_sel, ax= axs[0],label="DESI-Adjusted by formula", analytic_correction=True)
make_success_plot(cat_sel, ax= axs[0],label="DESI-Not adjusted", ref_sample=None, threshold=1.6)
# make_success_plot(cat_deep, ax= axs[0], label="DEEP-2/3", ls="--",  threshold=1.45)
# make_success_plot(cat_zcosmos, ax= axs[0],label="ZCOSMOS-Bright", ls="--")
axs[0].set_ylim(-0.1,1.1)
axs[0].set_title("25m to 1h15m", fontsize=20)
axs[0].legend(frameon=False)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())
# print(np.median(cat_sel["EXPTIME"]))

cat_sel = cat_desi[(cat_desi["EXPTIME"]>1.25*60) & (cat_desi["EXPTIME"]<=3.75*3600)]
make_success_plot(cat_sel, ax= axs[1], ref_sample=spec_cat,threshold=1.6)
make_success_plot(cat_sel, ax= axs[1], analytic_correction=True)
make_success_plot(cat_sel, ax= axs[1], ref_sample=None,threshold=1.6)
# make_success_plot(cat_deep, ax= axs[1], ls="--", threshold=1.45)
# make_success_plot(cat_zcosmos, ax= axs[1], ls="--")
axs[1].set_title("1h15m to 3h45m", fontsize=20)
axs[1].set_ylim(-0.1,1.1)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())
print(np.median(cat_sel["EXPTIME"]))

cat_sel = cat_desi[(cat_desi["EXPTIME"]>3.75*3600)]
make_success_plot(cat_sel, ax= axs[2], ref_sample=spec_cat, threshold=1.6)
make_success_plot(cat_sel, ax= axs[2], analytic_correction=True)
make_success_plot(cat_sel, ax= axs[2], ref_sample=None, threshold=1.6)

# make_success_plot(cat_deep, ax= axs[2], ls="--", threshold=1.45)
# make_success_plot(cat_zcosmos, ax= axs[2], ls="--")
axs[2].set_title("3h45m to 8h", fontsize=20)
axs[2].set_ylim(-0.1,1.1)
# print(cat_sel["EXPTIME"].min(),cat_sel["EXPTIME"].max())
print(np.median(cat_sel["EXPTIME"]))

axs[0].set_ylabel("Success Rate", fontsize=25)
axs[1].set_xlabel("$i$-mag", fontsize=25)
